In [4]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
# from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

#file setup
sdir=''
filename='Polarities.xlsx'
sheet = "Ratings"
dfrt = pd.read_excel(sdir+filename, sheet_name=sheet)
sheet = "Ranges"
dfrn = pd.read_excel(sdir+filename, sheet_name=sheet)

display(dfrt)
display(dfrn)

attrs = dfrt.columns
# these columns must match the abbreviations on sheet two of the excel data
dfrt.columns = ['idea', 'ar', 'ei', 'sc', 'ci', 'tb', 'am', 'wr', 'oc', 'sm', 'js', 'notes']
display(dfrt)


Idea/Example  \
0                                        By the Book   
1                                        Firing Room   
2                                      Test Facility   
3                                               Tank   
4             Recommended Docs <br> During Authoring   
5  RFID-Tagged, Wristband Voice <br>Command Tech ...   

   Atomicity (1) - Redundancy (10)  Inefficiency (1) - Efficiency (10)  \
0                                5                                   5   
1                               10                                  10   
2                                2                                   7   
3                                2                                   2   
4                                8                                   8   
5                                8                                   6   

   Custom (1) - Standard (10)  Ad Hoc (1) - Contingency (10)  \
0                           6                              7   
1                           2                              2   
2                           5                              3   
3                           4                              5   
4                           7                              8   
5                           8                              7   

   Top down (1) - Bottom Up (10)  Manual (1) - Automatic (10)  \
0                              3                            5   
1                              8                            2   
2                              5                            3   
3                              2                            3   
4                              7                            7   
5                              5                            7   

   Wasteful (1) - Resourceful (10)  Obscurity (1) - Clarity (10)  \
0                                5                             5   
1                                1                            10   
2                                5                             3   
3                                2                             2   
4                                8                             6   
5                                7                             5   

   Singular (1) - Multimodal (10)  Jarring (1) - Seamless (10)  \
0                               5                            6   
1                              10                           10   
2                               7                            6   
3                               5                            3   
4                               3                            7   
5                               8                            7   

                                               Notes  
0                                                NaN  
1  Redundancy reflects the number of people <br>f...  
2                                                NaN  
3                                                NaN  
4  Redundancy: Docs' info becomes more<br> redund...  
5  Redundancy: Feedback on Docs is<br>logged and ...

Left        Right Abbv
0     Atomicity   Redundancy   ar
1  Inefficiency   Efficiency   ei
2        Custom     Standard   sc
3        Ad Hoc  Contingency   ci
4      Top Down    Bottom Up   tb
5        Manual    Automatic   am
6      Wasteful  Resourceful   wr
7     Obscurity      Clarity   oc
8      Singular   Multimodal   sm
9       Jarring     Seamless   js

idea  ar  ei  sc  ci  tb  am  \
0                                        By the Book   5   5   6   7   3   5   
1                                        Firing Room  10  10   2   2   8   2   
2                                      Test Facility   2   7   5   3   5   3   
3                                               Tank   2   2   4   5   2   3   
4             Recommended Docs <br> During Authoring   8   8   7   8   7   7   
5  RFID-Tagged, Wristband Voice <br>Command Tech ...   8   6   8   7   5   7   

   wr  oc  sm  js                                              notes  
0   5   5   5   6                                                NaN  
1   1  10  10  10  Redundancy reflects the number of people <br>f...  
2   5   3   7   6                                                NaN  
3   2   2   5   3                                                NaN  
4   8   6   3   7  Redundancy: Docs' info becomes more<br> redund...  
5   7   5   8   7  Redundancy: Feedback on Docs is<br>logged and ...

In [2]:
import math 
for index, row in dfrt.iterrows():    
    d = []
    
    idea= row['idea']
    notes= row['notes']
    lower_range=attrs[1:].str.split(' - ').str[0]
    upper_range=attrs[1:].str.split(' - ').str[1]
    
    stats = row.drop('idea').drop('notes')
    d.append(Scatterpolar(
      r = stats,
      theta = upper_range,
      fill = 'toself',
      hoverinfo='r',
      marker = dict(
        color = "#8090c7",
        symbol = "square",
        size = 8
      )
    ))

    annot=[]
    for num in range(0,len(stats)):
        
        rr=0.05 # polar coord radius of annotations
        xcenter=0.5 # cartesian shift to get to center of graph
        ycenter=0.5 # cartesian shift to get to center of graph
        yscale=2.4 # annotation need scaling along y because of graph shape
        th=(2*np.pi/len(stats))*num #2pi = 360; get angular increment in radians
        thd=((360/len(stats))*num) # get angular increment in degrees
#         print('angle: '+ str(thd))
#         print('x: ' + str(np.cos(th)))
#         print('y: ' + str(np.sin(th)))
        xanchor="left"
        yanchor="top"
        if num in range(3,8): #flip the ones that are upside down
            thd = thd + 180
            xanchor="right"
        if num in range(0,5):
            yanchor="bottom"
#         if num in [0,5]: # this would put the text directly over the axis line
#             yanchor="middle"
        if num in [2,4,7,9]: # because of anchors, every other box sits on the opposite side of the axis. correct manually
            th += (2*np.pi/len(stats))/2
        if num in [1,4,6,9]: # because of anchors, every other box sits on the opposite side of the axis. correct manually
            rr -=0.006

        # polar coordinate math, translate to cartesian for the annotations
        # x = r × cos( θ )
        # y = r × sin( θ )
        x = xcenter+(rr*np.cos(th))
        y = ycenter+(yscale*rr*np.sin(th)) # y needs an extra multiplier because of graph shape
        annot.append(  
            dict(
#                 text='<b>.</b>', #use to check circle shape if things get wonky
                text='<b>'+lower_range[num]+'</b>',
                x=x,
                y=y,
                xanchor = xanchor,
                yanchor = yanchor,
                borderpad=0,
                showarrow=False,
                textangle=360-thd,
                font = dict(color="rgba(0,0,0,0.4)")
            ))

    if (str(notes) != 'nan'):
        annot.append(
            dict(
                text="Notes: " + notes,
                x=-.1,
                y=0.3,
                showarrow=False,
                align='left'
            ))
     
    # add an annotation with the ratings
    ratings = ""
    ratingvals = "<br>"
    for index, row in dfrn.iterrows():
        ratings += dfrn.iloc[index]["Left"]+"/"
        ratings +=dfrn.iloc[index]["Right"]+":  " 
        ratings+= "<br>"
        ratingvals += str(stats[dfrn.iloc[index]["Abbv"]]) + "<br>"
    # Rating Labels    
    annot.append(
        dict(
            text="<b>Ratings (1/10):</b> <br>" + ratings,
            x=-.1,
            y=1,
            showarrow=False,
            align='left'
        ))
    # Rating Values, stacked neatly
    annot.append(
        dict(
            text=ratingvals,
            x=.07,
            y=.95507,
            showarrow=False,
            align='left'
        ))
    
    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, 10],
          tickangle=0,
#           angle=90,
        ),
        angularaxis = dict(
          visible = True,
        ),
      ),
      showlegend = False,
      annotations=annot,
      title=idea
    )

    fig = Figure(data=d, layout=layout)
    iplot(fig)

In [3]:
# create pairs of the ones you'd like to compare. should extend to longer lists as well
for pair in [[0,1], [0,5]]:
#     print(row)
    d = []
    
    lower_range=attrs[1:].str.split(' - ').str[0]
    upper_range=attrs[1:].str.split(' - ').str[1]
    title = ''
    for ind, idd in enumerate(pair):
        row = dfrt.iloc[idd]
        idea= row['idea']
        title += " vs " + idea if ind > 0 else idea
        stats = row.drop('idea').drop('notes')
        d.append(Scatterpolar(
          r = stats,
          theta = upper_range,
          fill = 'toself',
          hoverinfo='r',
          name=idea,
    #       connectgaps=True,
          marker = dict(
            symbol = "square",
            size = 8
          )
        ))
    

#     notes= row['notes']
    
    

    annot=[]
    for num in range(0,len(stats)):
        
        rr=0.05 # polar coord radius of annotations
    #     xcenter=0 # cartesian shift to get to center of graph
    #     ycenter=0 # cartesian shift to get to center of graph
    #     yscale=0 # annotation need scaling along y because of graph shape
    #     rr=0.03 # polar coord radius of annotations
        xcenter=0.5 # cartesian shift to get to center of graph
        ycenter=0.5 # cartesian shift to get to center of graph
        yscale=2.4 # annotation need scaling along y because of graph shape
        th=(2*np.pi/len(stats))*num #2pi = 360; get angular increment in radians
        thd=((360/len(stats))*num) # get angular increment in degrees
#         print('angle: '+ str(thd))
#         print('x: ' + str(np.cos(th)))
#         print('y: ' + str(np.sin(th)))
        xanchor="left"
        yanchor="top"
        if num in range(3,8): #flip the ones that are upside down
            thd = thd + 180
            xanchor="right"
        if num in range(0,5):
            yanchor="bottom"
#         if num in [0,5]: # this would put the text directly over the axis line
#             yanchor="middle"
        if num in [2,4,7,9]: # because of anchors, every other box sits on the opposite side of the axis. correct manually
            th += (2*np.pi/len(stats))/2
        if num in [1,4,6,9]: # because of anchors, every other box sits on the opposite side of the axis. correct manually
            rr -=0.006
#         if num == 4: #manual tweak
#             y=y+0.03

        # polar coordinate math, translate to cartesian for the annotations
        # x = r × cos( θ )
        # y = r × sin( θ )
        x = xcenter+(rr*np.cos(th))
        y = ycenter+(yscale*rr*np.sin(th)) # y needs an extra multiplier because of graph shape
        annot.append(  
            dict(
#                 text='<b>ab</b>',
                text='<b>'+lower_range[num]+'</b>',
                x=x,
                y=y,
                xanchor = xanchor,
                yanchor = yanchor,
#                 bgcolor="#888888",
#                 bordercolor="#000000",
                borderpad=0,
#                 xref="polar",
#                 yref="polar",
                showarrow=False,
                textangle=360-thd,
                font = dict(color="rgba(0,0,0,0.4)")
            ))
#     print(str(notes))
#     if (str(notes) != 'nan'):
#         annot.append(
#             dict(
#                 text="Notes: " + notes,
#                 x=-.1,
#                 y=0.3,
#                 showarrow=False,
#                 align='left'
#             ))
     
    # add an annotation with the ratings
#     ratings = ""
#     ratingvals = "<br>"
#     for index, row in dfrn.iterrows():
#         ratings += dfrn.iloc[index]["Left"]+"/"
#         ratings +=dfrn.iloc[index]["Right"]+":  " 
#         ratings+= "<br>"
#         ratingvals += str(stats[dfrn.iloc[index]["Abbv"]]) + "<br>"
#     # Rating Labels    
#     annot.append(
#         dict(
#             text="<b>Ratings (1/10):</b> <br>" + ratings,
#             x=-.1,
#             y=1,
#             showarrow=False,
#             align='left'
#         ))
#     # Rating Values, stacked neatly
#     annot.append(
#         dict(
#             text=ratingvals,
#             x=.07,
#             y=.95507,
#             showarrow=False,
#             align='left'
#         ))
    
    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, 10],
          tickangle=0,
#           angle=90,
        ),
        angularaxis = dict(
          visible = True,
        ),
      ),
      showlegend = True,
      annotations=annot,
      title=title
    )

    fig = Figure(data=d, layout=layout)
    iplot(fig)